In [75]:
import pandas as pd 
import re
import seaborn as sns 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [104]:
data_train=pd.read_csv('./Data/train.csv')
data_test=pd.read_csv('./Data/test.csv')
ids_test=data_test.Id

In [105]:
data_test=data_test.drop('Id',axis=1)

In [106]:
X=data_train.copy()

In [107]:
#Procesamiento
# Todo a km
X['running']=X['running'].apply(lambda x: float(x.split(' ')[0]) * 1.60934 if x.split(' ')[2]=='miles' else float(x.split(' ')[0]))
data_test['running']=data_test['running'].apply(lambda x: float(x.split(' ')[0]) * 1.60934 if x.split(' ')[2]=='miles' else float(x.split(' ')[0]))

#Sustituir status por la mediana de su categoria
meds={s :X[X['status']==s]['price'].median() for s in X['status'].unique()}

X['status']=X['status'].map(meds)
data_test['status']=data_test['status'].map(meds)

In [108]:
data=pd.get_dummies(X.drop('color',axis=1)).astype(float)
data_test=pd.get_dummies(data_test.drop('color',axis=1)).astype(float)

y=data['price']
X=data.drop('price',axis=1)

X, data_test = X.align(data_test, join='left', axis=1)
data_test = pd.get_dummies(data_test)
X, X_tdata_testest = X.align(data_test, join='left', axis=1)



In [109]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, test_size=0.2,random_state=40)

In [110]:
my_model_1 = CatBoostRegressor(loss_function='MAE',random_state=0)

# Fit the model

my_model_1.fit(X_train_full, y_train)

predictions_1 = my_model_1.predict(X_valid_full)

# Calculate MAE
mae_1 = mean_absolute_error(predictions_1, y_valid)

#Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)
mean_absolute_error(my_model_1.predict(X_train_full), y_train)

0:	learn: 4917.4440221	total: 1.6ms	remaining: 1.6s
1:	learn: 4841.5306574	total: 2.29ms	remaining: 1.15s
2:	learn: 4751.8661239	total: 2.9ms	remaining: 962ms
3:	learn: 4696.1907851	total: 3.45ms	remaining: 860ms
4:	learn: 4606.8822637	total: 4.05ms	remaining: 805ms
5:	learn: 4530.1046705	total: 4.69ms	remaining: 778ms
6:	learn: 4454.9174354	total: 5.25ms	remaining: 745ms
7:	learn: 4377.3445277	total: 5.81ms	remaining: 720ms
8:	learn: 4328.9242882	total: 6.42ms	remaining: 707ms
9:	learn: 4252.8835973	total: 6.96ms	remaining: 689ms
10:	learn: 4191.9832306	total: 7.45ms	remaining: 670ms
11:	learn: 4134.2210971	total: 7.97ms	remaining: 656ms
12:	learn: 4078.2379760	total: 8.48ms	remaining: 644ms
13:	learn: 4009.1682060	total: 8.97ms	remaining: 632ms
14:	learn: 3950.8731026	total: 9.5ms	remaining: 624ms
15:	learn: 3891.3549847	total: 9.97ms	remaining: 613ms
16:	learn: 3836.9144684	total: 10.5ms	remaining: 605ms
17:	learn: 3778.5132824	total: 11.3ms	remaining: 616ms
18:	learn: 3727.0593976	

1385.2678995508181

In [112]:
preds=my_model_1.predict(data_test)
output = pd.DataFrame({'Id': ids_test,
                        'Predicted_Price': preds})
output.to_csv('submission.csv', index=False)

In [114]:
#Resultados obtenidos
mod=['Catboost sin color usando medianas para sustituir status']
train=[1385.27]
test=[1979.31]
externa=[1834.16]

data=pd.DataFrame(zip(mod,train,test,externa),columns=['Modelo','MAE TRAIN','MAE TEST','MAE PUBLICO'])
data.head()

,Modelo,MAE TRAIN,MAE TEST,MAE PUBLICO
0,Catboost sin color usando medianas para sustit...,1385.27,1979.31,1834.16
